### Tests for Applying Gains Matrix

In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))
print(parent_dir)

/home/mike/corrcal_gpu_pipeline/pipeline


In [2]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *
from invcov import *

hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.


In [3]:
apply_gains_to_mat?
# undo_zeropad?

Signature:
apply_gains_to_mat(
    gains: cupy.ndarray,
    mat: cupy.ndarray,
    edges: cupy.ndarray,
    ant_1_array: cupy.ndarray,
    ant_2_array: cupy.ndarray,
    xp: Any,
    is_zeropadded: bool = True,
)
Docstring:
Apply a pair of complex gains to a matrix. Utilizes the Re/Im split.
Only accounts for "one half" of the gain application, meaning the 
function is really performing eg. (g_1g_2*\Delta_{1,2}), where it is 
understood that antenna's 1 and 2 belong to the baseline sitting at
the same row as that baseline row in the \Delta (\Sigma) matrix. Note that 
although the matrix provided may be zeropadded, the gain matrix is zeropadded 
here and as such, should always be provided in an un-zeropadded (original) format.

NOTE: Could be smart in the future to wrap the zeropadding of the gain 
    matrix in a separate function.

Params
------
gains
    1D array of Re/Im alternating gains to be applied to the source or
    diffuse matrices. Contains one set of Re/Im gains for all an

In [4]:
summarize_benchmark_results?

Signature: summarize_benchmark_results(function, *args)
Docstring:
Use the CuPy benchmark function and nicely print out just the
total time spent on both the gpu and the cpu.
File:      ~/corrcal_gpu_pipeline/pipeline/utils.py
Type:      function

In [11]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 4
n_bl = 2*n_ant**2
n_gains = 2*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
# ant_1_array = cp.arange(n_bl//2)
# ant_2_array = cp.arange(n_bl//2, 2*n_bl//2)

ant_1_upper, ant_2_upper = cp.triu_indices(n_ant, k=1)
ant_1_lower, ant_2_lower = cp.tril_indices(n_ant, k=-1)
ant_1_array = cp.concatenate((ant_1_upper, ant_1_lower))
ant_2_array = cp.concatenate((ant_2_upper, ant_2_lower))
print(ant_1_array)
print(ant_2_array)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  4  6 16 28 32]
[0 0 0 1 1 2 1 2 2 3 3 3]
[1 2 3 2 3 3 0 0 1 0 1 2]


In [12]:
zp_cplex_gain_mat = zeropad_gains(sim_gains, edges, ant_1_array, ant_2_array, xp = cp, return_inv=False)
gain_diff_mat = apply_gains(zp_cplex_gain_mat, sim_diff_mat_3d, xp=cp)
# print(gain_diff_mat)

In [7]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

/home/mike/corrcal_gpu_pipeline/pipeline/utils.py:145: SyntaxWarning: invalid escape sequence '\D'
  """


ValueError: operands could not be broadcast together with shapes (12, 1) (2, 1)

In [ ]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (4, 8, 3)

Gains applied mat has shape (4, 8, 3)

The mat with gains applied looks like:

[[[0.1579835  0.50598987 0.94385764]
  [1.03429943 0.2482692  0.82212146]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.41185653 0.41826525 0.46022986]
  [0.00342292 0.23704531 0.09259659]
  [0.40371477 0.59672464 0.67435857]
  [0.22702284 0.80834479 0.84761636]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.14617583 0.05804618 0.29960561]
  [0.05887869 0.27587957 0.13282545]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        

#### Random Debugging tests

In [ ]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[0.27771907 0.49786286 0.78252181]
  [0.84870461 0.05388455 0.30331298]
  [0.25126195 0.81340359 0.90224821]
  [0.01936672 0.79699927 0.67417065]
  [0.28822179 0.09869691 0.65989059]]

 [[0.08421076 0.26292774 0.04935195]
  [0.10100533 0.95292719 0.39047679]
  [0.41441608 0.06360632 0.71426598]
  [0.58761828 0.57467898 0.32634538]
  [0.93276915 0.38334206 0.51171445]]]
[[[0.27771907 0.49786286 0.78252181]
  [0.25126195 0.81340359 0.90224821]
  [0.28822179 0.09869691 0.65989059]]

 [[0.08421076 0.26292774 0.04935195]
  [0.41441608 0.06360632 0.71426598]
  [0.93276915 0.38334206 0.51171445]]]


In [ ]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.66436983 0.72524934 0.16675385 0.14482441 0.22502036 0.61383921
 0.16619552 0.99967343 0.72010916 0.69081126]
[0.66436983 0.72524934 0.16675385 0.14482441 0.22502036]

[0.61383921 0.16619552 0.99967343 0.72010916 0.69081126]

[0.66436983 0.72524934 0.16675385 0.14482441 0.22502036 0.61383921
 0.16619552 0.99967343 0.72010916 0.69081126]

[0.66436983 0.72524934 0.16675385 0.14482441 0.22502036]
[0.61383921 0.16619552 0.99967343 0.72010916 0.69081126]


#### Check ZP computes the same thing as corrcal

In [ ]:
#Current corrcal gain application function
def apply_gains_to_mat_corrcal(gains, mat, ant_1_array, ant_2_array):
    """Apply a gain-like matrix to a provided matrix."""
    complex_gains = gains[::2] + 1j*gains[1::2]
    gain_mat = (
        complex_gains[ant_1_array,None] * complex_gains[ant_2_array,None].conj()
    )
    out = np.zeros_like(mat)
    out[::2] = gain_mat.real * mat[::2] - gain_mat.imag * mat[1::2]
    out[1::2] = gain_mat.imag * mat[::2] + gain_mat.real * mat[1::2]
    return out

In [ ]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)


The edges of the redundant blocks have indices[ 0 18 20 24 30]


In [ ]:
out_corrcal = apply_gains_to_mat_corrcal(sim_gains, sim_diff_mat, ant_1_array, ant_2_array)
out_gpu = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, cp)
out_gpu_corrcal = apply_gains_to_mat(sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, cp, False)
out_gpu_resh = undo_zeropad(out_gpu, edges, cp)

In [ ]:
#should return True if my routine computes the same thing as the current corrcal routine
print(cp.allclose(out_gpu_resh, out_gpu_corrcal))
print(cp.allclose(out_gpu_resh, out_corrcal))

True
True


#### Timing test for np vs cp versions

In [ ]:
#simulation params relevant for testing application of gains to a matrix
n_bl = 240000  # something like 2*N_ant^2 for Re/Im split
n_ant = 512
n_gains = 2*n_bl  #one for each antenna and one for each Re and Im part
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
if xp == cp:

    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
    sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant

    #leave as n_bl/2 since indexex into cplex gains which are not Re/Im split
    ant_1_array = cp.arange(n_bl//2) 
    ant_2_array = cp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

elif xp == np:
    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig).astype('float64')
    sim_gains = xp.random.rand(n_gains).astype('float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    ant_1_array = xp.arange(n_bl//2)
    ant_2_array = xp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    # zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    # sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

print(len(edges))

The edges of the redundant blocks have indices[     0    388   1100   1834   2370   2602   3782   4096   4390   5618
   5712   6724   7284   7910   7932   7970   8162   8508   8950   9024
   9982  10278  10380  10464  10976  11042  11234  12476  12616  12732
  13182  13896  14096  14504  14564  14578  14970  14984  15020  15750
  16194  16276  16902  17196  17310  17846  18096  18254  18580  18630
  20274  21474  21790  22424  22638  24118  25802  25864  26736  26836
  27212  27302  27798  28092  28778  29668  30806  31038  31068  31498
  31666  31980  34152  34260  34784  35600  35674  35816  36100  37368
  38154  38262  38730  39900  40210  40402  40698  40766  41234  41538
  41884  42546  42766  43572  43774  43810  44104  44154  44318  44546
  45316  45864  45998  46256  46884  46898  48562  48860  49022  49470
  49708  49832  49840  50446  51014  52096  52538  53190  53310  53470
  53626  54438  54466  56264  56294  56474  56970  57394  57712  58278
  58602  59044  59174  59596  6

In [ ]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, xp, False)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>


Time on cpu: 0.000878s
Time on gpu: 0.000908s


In [ ]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, xp, True)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.002599s
Time on gpu: 0.003795s


### Random debugging....

In [ ]:
import numpy as np
mat = np.random.rand(3, 2, 3)
print(mat)
gm = np.arange(6)
gm = gm.reshape(3, 2, 1)
print(gm)
print()
print(gm * mat)

[[[0.51492247 0.36418569 0.631089  ]
  [0.58425926 0.02913226 0.70991368]]

 [[0.39597714 0.28708475 0.32907441]
  [0.71888024 0.97021329 0.72432182]]

 [[0.47422192 0.91443269 0.99782788]
  [0.27146681 0.55374009 0.42369466]]]
[[[0]
  [1]]

 [[2]
  [3]]

 [[4]
  [5]]]

[[[0.         0.         0.        ]
  [0.58425926 0.02913226 0.70991368]]

 [[0.79195429 0.57416949 0.65814883]
  [2.15664073 2.91063986 2.17296546]]

 [[1.89688766 3.65773078 3.99131152]
  [1.35733407 2.76870046 2.11847329]]]


In [ ]:
a = np.random.rand(4,4)
print(a)
print(a[::2])

[[0.38798259 0.33793608 0.51954081 0.93072696]
 [0.79241774 0.45451251 0.68514549 0.30767765]
 [0.69755979 0.94056492 0.05684167 0.81831362]
 [0.23845743 0.09610608 0.9587009  0.30306608]]
[[0.38798259 0.33793608 0.51954081 0.93072696]
 [0.69755979 0.94056492 0.05684167 0.81831362]]


In [ ]:
a = np.arange(15)
b = np.arange(15)

print(a[:, None]* b[:, None])

[[  0]
 [  1]
 [  4]
 [  9]
 [ 16]
 [ 25]
 [ 36]
 [ 49]
 [ 64]
 [ 81]
 [100]
 [121]
 [144]
 [169]
 [196]]
